In [7]:
import numpy as np
from collections import defaultdict
import pandas as pd
from functools import reduce
import glob
import torch
# import matplotlib.pyplot as plt


In [8]:
result_collect_mae = defaultdict(dict)
result_collect_mse = defaultdict(dict)

models = ["contrast-dtcn-moco", ]


In [26]:
# sorted_model = []
# for model in models:
#     for predleng in [24, 48, 168, 336, 720]:
#         sorted_model.append(model+"_" + str(predleng))
        
sorted_out_task = ["ETTh1", "ETTm1", "ECL"]  #, "ETTh2"

In [27]:
def visual_itera(mse_loss, cl_loss, save_path):
    plt.figure()
    plt.plot(mse_loss, label='MSE loss', linewidth=2)
    if len(cl_loss) > 0:
        plt.plot(cl_loss, label='CL loss', linewidth=2)
    plt.legend()
    plt.savefig(save_path, bbox_inches='tight')
def visual_epoch(train_loss, val_loss, test_loss, save_path):
    plt.figure()
    plt.plot(train_loss, label='Training loss', linewidth=2)
    plt.plot(val_loss, label='Validation loss', linewidth=2)
    plt.plot(test_loss, label='Test loss', linewidth=2)
    plt.legend()
    plt.savefig(save_path, bbox_inches='tight')

In [28]:
paths 

['./twophase_res/contrast-lstm-moco_ETTh1_contrastL0.5_freezeenTrue_svmFalse_mareFalse_mask0.3_l2normTrue_ftM_sl168_pl168_dm320_el5_avgFalse_cosTrue_augcost_Exp_2',
 './twophase_res/contrast-lstm-moco_ETTh1_contrastL0.5_freezeenTrue_svmFalse_mareFalse_mask0.3_l2normTrue_ftM_sl48_pl24_dm320_el5_avgFalse_cosTrue_augcost_Exp_2',
 './twophase_res/contrast-lstm-moco_ETTh1_contrastL0.5_freezeenTrue_svmFalse_mareFalse_mask0.3_l2normTrue_ftM_sl720_pl720_dm320_el5_avgFalse_cosTrue_augcost_Exp_1',
 './twophase_res/contrast-lstm-moco_ETTh1_contrastL0.5_freezeenTrue_svmFalse_mareFalse_mask0.3_l2normTrue_ftM_sl96_pl48_dm320_el5_avgFalse_cosTrue_augcost_Exp_1',
 './twophase_res/contrast-lstm-moco_ETTm1_contrastL0.5_freezeenTrue_svmFalse_mareFalse_mask0.3_l2normTrue_ftM_sl384_pl96_dm320_el5_avgFalse_cosTrue_augcost_Exp_2',
 './twophase_res/contrast-lstm-moco_ETTh2_contrastL0.5_freezeenTrue_svmFalse_mareFalse_mask0.3_l2normTrue_ftM_sl720_pl720_dm320_el5_avgFalse_cosTrue_augcost_Exp_0',
 './twophase_re

In [38]:
all_df = []
root_path = "./twophase_res/"
var_res = "multivar"
weight = "0.0"
freeze = "freezeenFalse"
vis = False
for weight in ["0.5"]:
    for model in ["-tcn-moco_", "-dtcn-moco_", "-lstm-moco_", "-informer-moco_"]: #"lstm", "tcn"
        print(model)
        mask_rule = model

        paths = glob.glob(root_path + "/*")

        paths = [x for x in paths if "csv" not in x]
        paths = [x for x in paths if model in x]
        paths = [x for x in paths if freeze in x]
#         paths = [x for x in paths if "svmTrue" in x]
#         paths = [x for x in paths if "avgFalse" in x]
        paths = [x for x in paths if "augcost" in x]

        result_collect_mae = defaultdict(dict)
        result_collect_mse = defaultdict(dict)

        for p in paths:
            file = p.split("/")[2]

            if "ftS" in file:
                var_type = "univar"
            else:
                var_type = "multivar"

            if var_type == var_res:
                pl = int(file.split("_pl")[1].split("_")[0])
                data = file.split("_")[1]
                try:
                    metric = np.load(p + "/metrics.npy")

                    mae = metric[0]
                    mse = metric[1]

                    if pl in result_collect_mae[data].keys():
                        result_collect_mae[data][pl].append(mae)
                        result_collect_mse[data][pl].append(mse)
                    else:
                        result_collect_mae[data][pl] = [mae]
                        result_collect_mse[data][pl] = [mse]

                    if "Exp_0" in p:
                        loggs = torch.load(p + "/training_log.pt")
                        itera_train_loss_mse = loggs["itera_train_loss_mse"]
                        itera_train_loss_cl = loggs["itera_train_loss_cl"]
                        epoch_train_loss = loggs["epoch_train_loss"]
                        epo_eval_loss = loggs["epo_eval_loss"]
                        epo_test_loss = loggs["epo_test_loss"]
                        num_step = loggs["num_step"]
                        total_step = len(itera_train_loss_mse)

                        if vis:
                            visual_itera(itera_train_loss_mse, itera_train_loss_cl, "./hyper_results/visualization/iteration_"+p.split("/")[-1]+".jpg")
                            visual_itera(itera_train_loss_mse[int(total_step*0.1):], itera_train_loss_cl[int(total_step*0.1):], "./hyper_results/visualization/zoom_iteration_"+p.split("/")[-1]+".jpg")
                            visual_epoch(epoch_train_loss, epo_eval_loss, epo_test_loss, "./hyper_results/visualization/epoch_"+p.split("/")[-1]+".jpg")

                except:
#                     print(p)
                    continue

        for data in result_collect_mae.keys():
            for pl in result_collect_mae[data].keys():
                print(data,pl)
                print(len(result_collect_mae[data][pl]))
        #         print(result_collect_mae[data][pl])

                result_collect_mae[data][pl] = sum(result_collect_mae[data][pl]) / len(result_collect_mae[data][pl])
                result_collect_mse[data][pl] = sum(result_collect_mse[data][pl]) / len(result_collect_mse[data][pl])

        result_mae = []
        result_mse = []
        for data in sorted_out_task:
            for pl in result_collect_mae[data].keys():

                result_mae.append([data+"-"+str(pl).zfill(3), result_collect_mae[data][pl]])
                result_mse.append([data+"-"+str(pl).zfill(3), result_collect_mse[data][pl]])


        res_df_mae = pd.DataFrame(result_mae, columns = ["data", "mae_"+model+weight]).sort_values(by="data")
        res_df_mse = pd.DataFrame(result_mse, columns = ["data", "mse_"+model+weight]).sort_values(by="data")

        res_df = res_df_mse.merge(res_df_mae)

        all_df.append(res_df)

-tcn-moco_
ETTm1 96
3
ETTm1 24
3
ETTm1 48
3
ETTm1 672
3
ETTm1 288
3
ETTh1 168
3
ETTh1 720
3
ETTh1 336
3
ETTh1 48
3
ETTh1 24
3
ECL 168
3
ECL 48
3
ECL 24
3
ECL 720
3
ECL 336
3
ETTh2 24
3
ETTh2 336
3
ETTh2 720
3
ETTh2 48
3
ETTh2 168
3
-dtcn-moco_
ECL 24
3
ECL 168
3
ECL 336
3
ECL 720
3
ECL 48
3
ETTh1 168
3
ETTh1 48
3
ETTh1 720
3
ETTh1 336
3
ETTh1 24
3
ETTm1 48
3
ETTm1 288
3
ETTm1 24
3
ETTm1 672
3
ETTm1 96
3
ETTh2 24
3
ETTh2 168
3
ETTh2 48
3
ETTh2 720
3
ETTh2 336
3
-lstm-moco_
ETTh2 720
3
ETTh2 48
3
ETTh2 336
3
ETTh2 168
3
ETTh2 24
3
ECL 336
3
ECL 48
3
ECL 168
3
ECL 24
3
ECL 720
3
ETTh1 48
3
ETTh1 24
3
ETTh1 336
3
ETTh1 168
3
ETTh1 720
3
ETTm1 288
3
ETTm1 672
3
ETTm1 48
3
ETTm1 24
3
ETTm1 96
3
-informer-moco_
ECL 48
3
ECL 24
3
ECL 720
3
ECL 336
3
ECL 168
3
ETTh2 336
3
ETTh2 48
3
ETTh2 24
3
ETTh2 168
3
ETTh2 720
3
ETTm1 24
3
ETTm1 96
3
ETTm1 288
3
ETTm1 672
3
ETTm1 48
3
ETTh1 720
3
ETTh1 48
3
ETTh1 336
3
ETTh1 168
3
ETTh1 24
3


In [39]:
final_df = reduce(lambda df1,df2: pd.merge(df1,df2,on='data',how="outer"), all_df)
final_df.loc[final_df.shape[0]] = final_df.mean()
final_df["data"][final_df.shape[0]-1] = "mean"
final_df.to_csv("./" + freeze +"_twophase_avg.csv")

/tmp/ipykernel_1025851/3764835458.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["data"][final_df.shape[0]-1] = "mean"


In [40]:
final_df

,data,mse_-tcn-moco_0.5,mae_-tcn-moco_0.5,mse_-dtcn-moco_0.5,mae_-dtcn-moco_0.5,mse_-lstm-moco_0.5,mae_-lstm-moco_0.5,mse_-informer-moco_0.5,mae_-informer-moco_0.5
0,ECL-024,0.270056,0.355863,0.232293,0.338976,0.301429,0.371791,0.259306,0.353925
1,ECL-048,0.304399,0.378743,0.260366,0.354009,0.388039,0.424058,0.291294,0.372454
2,ECL-168,0.304901,0.382533,0.289096,0.376807,0.324686,0.391332,0.267351,0.362488
3,ECL-336,0.297102,0.375418,0.285424,0.368919,0.375770,0.422296,0.284107,0.376659
4,ECL-720,0.353873,0.414769,0.311148,0.389327,0.397574,0.431377,0.365324,0.419317
5,ETTh1-024,0.535626,0.525909,0.501599,0.505709,0.990925,0.799869,0.547431,0.546031
6,ETTh1-048,0.528514,0.529949,0.805179,0.681035,1.063487,0.835506,0.848490,0.719581
7,ETTh1-168,0.822963,0.701536,1.007199,0.776688,1.059961,0.822875,0.947136,0.725896
8,ETTh1-336,0.929625,0.724860,1.177743,0.862066,1.052628,0.803723,1.072824,0.796518
9,ETTh1-720,0.895580,0.739595,1.289357,0.892435,1.183118,0.865116,0.937712,0.765258


In [19]:
loggs.keys()

dict_keys(['itera_train_loss_mse', 'itera_train_loss_cl', 'epoch_train_loss', 'epo_eval_loss', 'epo_test_loss', 'num_step'])

In [4]:
# dfs_mae = []
# dfs_mse = []

# for run in [0, 1, 2]:
#     for predleng in [24, 48, 168, 336, 720]:
#         for data in ["ETTh1"]:#["ECL", "ETTm2", "Exchange", "traffic", "weather"]:
#             for model in models:
#                 if data == "ECL":
#                     file_name = "custom"
#                 else:
#                     file_name = data
                    
#                 folder_name = "./results/{}_{}_contrastL0.5_ftM_s168_ll68_pl{}_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_Exp_{}".format( model, data, file_name, str(predleng), str(run))
                
# #                 try:
#                 metric = np.load(folder_name + "/metrics.npy")
#                 mae = metric[0]
#                 mse = metric[1]
#                 result_collect_mae[data][model+"_" + str(predleng)] = mae
#                 result_collect_mse[data][model+"_" + str(predleng)] = mse
# #                 except:
# #                 print(folder_name)
# #                 break
    
#     result_mae = np.zeros((len(sorted_out_task), len(sorted_model)))
#     result_mse = np.zeros((len(sorted_out_task), len(sorted_model)))

#     for i, task in enumerate(sorted_out_task):
#         for j, model in enumerate(sorted_model):
#             try:
#                 result_mae[i][j] = result_collect_mae[task][model]
#                 result_mse[i][j] = result_collect_mse[task][model]
#             except:
#                 pass

#     res_df_mae = pd.DataFrame(result_mae, columns=sorted_model)
#     res_df_mse = pd.DataFrame(result_mse, columns=sorted_model)
#     # res_df["best_model"] = res_df.idxmax(axis=1)

# #     res_df.loc['mean'] = res_df.mean()
#     res_df_mae.insert(loc=0, column='task', value=sorted_out_task)
#     res_df_mse.insert(loc=0, column='task', value=sorted_out_task)
#     dfs_mae.append(res_df_mae)
#     dfs_mse.append(res_df_mse)
            

In [5]:
# df_concat_mae = pd.concat(dfs_mae)
# df_avg_mae = df_concat_mae.groupby(level=0).mean()

# df_avg_mae.insert(loc=0, column='task', value=sorted_out_task)
# df_avg_mae.to_csv("./avg_mae.csv", index=False)

In [120]:
# df_concat_mse = pd.concat(dfs_mse)
# df_avg_mse = df_concat_mse.groupby(level=0).mean()

# df_avg_mse.insert(loc=0, column='task', value=sorted_out_task)
# df_avg_mse.to_csv("./avg_mse.csv", index=False)

In [121]:
# df_avg_mae

In [122]:
# df_concat_mae = pd.concat(dfs_mae)
# df_std_mae = df_concat_mae.groupby(level=0).std()

# df_std_mae.insert(loc=0, column='task', value=sorted_out_task)
# df_std_mae.to_csv("./std_mae.csv", index=False)

In [123]:
# df_concat_mse = pd.concat(dfs_mse)
# df_std_mse = df_concat_mse.groupby(level=0).std()

# df_std_mse.insert(loc=0, column='task', value=sorted_out_task)
# df_std_mse.to_csv("./std_mse.csv", index=False)

In [124]:
# df_std_mae

In [125]:
# for run in [0, 1, 2]:
#     for predleng in [24, 48, 168, 336, 720]:
#         for data in ["ETTh1"]:#["ECL", "ETTm2", "Exchange", "traffic", "weather"]:
#             for model in models:
#                 if data == "ECL":
#                     file_name = "custom"
#                 else:
#                     file_name = data
                    
#                 folder_name = "./results/{}_{}_contrastL0.5_ftM_s168_ll68_pl{}_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_Exp_{}".format( model, data, file_name, str(predleng), str(run))
                
# #                 try:
#                 metric = np.load(folder_name + "/metrics.npy")
#                 mae = metric[0]
#                 mse = metric[1]
#                 result_collect_mae[data][model+"_" + str(predleng)] = mae
#                 result_collect_mse[data][model+"_" + str(predleng)] = mse
# #                 except:
# #                 print(folder_name)
# #                 break
    
#     result_mae = np.zeros((len(sorted_out_task), len(sorted_model)))
#     result_mse = np.zeros((len(sorted_out_task), len(sorted_model)))

#     for i, task in enumerate(sorted_out_task):
#         for j, model in enumerate(sorted_model):
#             try:
#                 result_mae[i][j] = result_collect_mae[task][model]
#                 result_mse[i][j] = result_collect_mse[task][model]
#             except:
#                 pass

#     res_df_mae = pd.DataFrame(result_mae, columns=sorted_model)
#     res_df_mse = pd.DataFrame(result_mse, columns=sorted_model)
#     # res_df["best_model"] = res_df.idxmax(axis=1)

# #     res_df.loc['mean'] = res_df.mean()
#     res_df_mae.insert(loc=0, column='task', value=sorted_out_task)
#     res_df_mse.insert(loc=0, column='task', value=sorted_out_task)
#     dfs_mae.append(res_df_mae)
#     dfs_mse.append(res_df_mse)
            

In [13]:
# res_df_mse.to_csv("./hyper_results/"+var_res+mask_rule+"avg_mse.csv")
# res_df_mae.to_csv("./hyper_results/"+var_res+mask_rule+"avg_mae.csv")
# res_df.to_csv("./hyper_results/"+var_res+mask_rule+"avg.csv")